In [1]:
# Imports required libraries
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import hvplot
from pathlib import Path

%matplotlib inline

In [2]:
load_dotenv()

True

In [3]:
# Locates and stores keys
alpaca_api = os.getenv("ALPACA_API_KEY")
alpaca_secret = os.getenv("ALPACA_SECRET_KEY")
type(alpaca_api)
type(alpaca_secret)

# Creates the Alpaca tradeapi.REST object
alpaca = tradeapi.REST(
    alpaca_api,
    alpaca_secret,
    api_version="v2"
)

In [4]:
# Sets timeframe to 1Day
timeframe = "1Day"

# Formats current date as ISO format
# You can choose to set both the start and end date at the date of your prior weekday 
# This will give you the closing price of the previous trading day
# Alternatively you could select any arbitrary start/end dates
start_date = pd.Timestamp("2022-05-10", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2022-05-10", tz="America/New_York").isoformat()


In [5]:
# Creates the list of tickers that the user will input, and allows for their input

tickers = []

tickers.append(input("Enter your first stock or bond ticker (as it appears on the exchange (example: BBBY)) "))
tickers.append(input("Enter your second ticker."))
tickers.append(input("Enter your third ticker."))
tickers.append(input("Enter your fourth ticker."))
tickers.append(input("Enter your fifth ticker."))
print(f"The tickers you have selected are: {tickers}")


Enter your first stock or bond ticker (as it appears on the exchange (example: BBBY))  GME
Enter your second ticker. AMC
Enter your third ticker. MSFT
Enter your fourth ticker. GOOG
Enter your fifth ticker. AMZN


The tickers you have selected are: ['GME', 'AMC', 'MSFT', 'GOOG', 'AMZN']


In [6]:
# Uses the Alpaca get_bars function to get current closing prices each ticker in
# the portfolio and creates a dataframe
portfolio = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df
    
# Reorganizes the DataFrame
# Separates ticker data
ticker_1 = portfolio[portfolio['symbol']==portfolio.iloc[:,7]].drop('symbol', axis=1)
ticker_2 = portfolio[portfolio['symbol']==portfolio.iloc[:,7]].drop('symbol', axis=1)
ticker_3 = portfolio[portfolio['symbol']==portfolio.iloc[:,7]].drop('symbol', axis=1)
ticker_4 = portfolio[portfolio['symbol']==portfolio.iloc[:,7]].drop('symbol', axis=1)
ticker_5 = portfolio[portfolio['symbol']==portfolio.iloc[:,7]].drop('symbol', axis=1)

# Concatenates the ticker DataFrames
portfolio_df = pd.concat(
    [ticker_1, ticker_2, ticker_3, ticker_4, ticker_5],
    axis=1,
    keys=[tickers[0], tickers[1], tickers[2], tickers[3], tickers[4]]
)

# Reviews the Alpaca DataFrame
portfolio_df.head()

GME                                         \
                              open     high        low    close    volume   
timestamp                                                                   
2022-05-10 04:00:00+00:00    13.15    14.01    11.5000    11.84  55514689   
2022-05-10 04:00:00+00:00  2225.00  2252.85  2143.4200  2177.18   5244153   
2022-05-10 04:00:00+00:00   102.98   106.06    88.0000    93.50   4131403   
2022-05-10 04:00:00+00:00  2320.81  2333.82  2267.6657  2291.69   1544004   
2022-05-10 04:00:00+00:00   271.67   273.75   265.0700   269.50  39056986   

                                                        AMC           \
                          trade_count         vwap     open     high   
timestamp                                                              
2022-05-10 04:00:00+00:00      341550    12.370858    13.15    14.01   
2022-05-10 04:00:00+00:00      310000  2188.168735  2225.00  2252.85   
2022-05-10 04:00:00+00:00      100311    95.751817   102.98   106.06   
2022-05-10 04:00:00+00:00       95597  2302.228626  2320.81  2333.82   
2022-05-10 04:00:00+00:00      521301   269.871368   271.67   273.75   

                                      ...      GOOG                           \
                                 low  ...    volume trade_count         vwap   
timestamp                             ...                                      
2022-05-10 04:00:00+00:00    11.5000  ...  55514689      341550    12.370858   
2022-05-10 04:00:00+00:00  2143.4200  ...   5244153      310000  2188.168735   
2022-05-10 04:00:00+00:00    88.0000  ...   4131403      100311    95.751817   
2022-05-10 04:00:00+00:00  2267.6657  ...   1544004       95597  2302.228626   
2022-05-10 04:00:00+00:00   265.0700  ...  39056986      521301   269.871368   

                              AMZN                                         \
                              open     high        low    close    volume   
timestamp                                                                   
2022-05-10 04:00:00+00:00    13.15    14.01    11.5000    11.84  55514689   
2022-05-10 04:00:00+00:00  2225.00  2252.85  2143.4200  2177.18   5244153   
2022-05-10 04:00:00+00:00   102.98   106.06    88.0000    93.50   4131403   
2022-05-10 04:00:00+00:00  2320.81  2333.82  2267.6657  2291.69   1544004   
2022-05-10 04:00:00+00:00   271.67   273.75   265.0700   269.50  39056986   

                                                    
                          trade_count         vwap  
timestamp                                           
2022-05-10 04:00:00+00:00      341550    12.370858  
2022-05-10 04:00:00+00:00      310000  2188.168735  
2022-05-10 04:00:00+00:00      100311    95.751817  
2022-05-10 04:00:00+00:00       95597  2302.228626  
2022-05-10 04:00:00+00:00      521301   269.871368  

[5 rows x 35 columns]

In [21]:
# Allows the user to input the quantity of their stocks or bonds
ticker_1_qty = float(input(f"How many shares of {tickers[0]} do you own?"))
ticker_2_qty = float(input(f"How many shares of {tickers[1]} do you own?"))
ticker_3_qty = float(input(f"How many shares of {tickers[2]} do you own?"))
ticker_4_qty = float(input(f"How many shares of {tickers[3]} do you own?"))
ticker_5_qty = float(input(f"How many shares of {tickers[4]} do you own?"))


How many shares of GME do you own? 2
How many shares of AMC do you own? 2
How many shares of MSFT do you own? 2
How many shares of GOOG do you own? 2
How many shares of AMZN do you own? 2


In [22]:
ticker_1_close = float(portfolio_df.iat[0,3])
ticker_2_close = float(portfolio_df.iat[1,3])
ticker_3_close = float(portfolio_df.iat[2,3])
ticker_4_close = float(portfolio_df.iat[3,3])
ticker_5_close = float(portfolio_df.iat[4,3])
                                    
print(f"The closing price of {tickers[0]} was {ticker_1_close}")
print(f"The closing price of {tickers[1]} was {ticker_2_close}")
print(f"The closing price of {tickers[2]} was {ticker_3_close}")
print(f"The closing price of {tickers[3]} was {ticker_4_close}")
print(f"The closing price of {tickers[4]} was {ticker_5_close}")

The closing price of GME was 11.84
The closing price of AMC was 2177.18
The closing price of MSFT was 93.5
The closing price of GOOG was 2291.69
The closing price of AMZN was 269.5


In [24]:
ticker_1_value = ticker_1_qty * ticker_1_close
ticker_2_value = ticker_2_qty * ticker_2_close
ticker_3_value = ticker_3_qty * ticker_3_close
ticker_4_value = ticker_4_qty * ticker_3_close
ticker_5_value = ticker_5_qty * ticker_3_close
print(ticker_1_value, ticker_2_value, ticker_3_value, ticker_4_value, ticker_5_value)

23.68 4354.36 187.0 187.0 187.0


In [ ]:
# Reads the S&P 500 data into a dataframe
sp500 = pd.read_csv(
    Path("Resources/Download Data - INDEX_US_S&P US_SPX.csv"),
    index_col="Date",
    parse_dates=True,
    infer_datetime_format=True
)

In [ ]:
# Displays the data
display(sp500.head())
display(sp500.tail())

In [ ]:
# Checks for null values in the data
sp500.isnull().sum()

In [ ]:
# Cleans and prepares the data for the dataframe
sp500.loc[:, 'Open'] = sp500.loc[:, 'Open'].str.replace(",", "")
sp500.loc[:, 'High'] = sp500.loc[:, 'High'].str.replace(",", "")
sp500.loc[:, 'Low'] = sp500.loc[:, 'Low'].str.replace(",", "")
sp500.loc[:, 'Close'] = sp500.loc[:, 'Close'].str.replace(",", "")

# Confirms cleaning of data
sp500.head()

In [ ]:
# Changes all data types to float
sp500.loc[:, 'Open'] = sp500.loc[:, 'Open'].astype('float')
sp500.loc[:, 'High'] = sp500.loc[:, 'High'].astype('float')
sp500.loc[:, 'Low'] = sp500.loc[:, 'Low'].astype('float')
sp500.loc[:, 'Close'] = sp500.loc[:, 'Close'].astype('float')

# Confirms data type conversion
sp500.dtypes